# Batch

## JSON_object

In [ ]:
data = {
    "custom_id": str(id),
    "method": "POST",
    "url": "/v1/chat/completions",
    "body": {
        "model": "gpt-3.5-turbo-0125",
        "messages": messages,
        "response_format": { "type": "json_object" },
        "max_tokens": 500,
    }
}

## JSON_schema

In [ ]:
response_format={
  "type": "json_schema",
  "json_schema": {
    "name": "user_personality_analysis",
    "strict": True,
    "schema": {
      "type": "object",
      "properties": {
        "dimensions": {
          "type": "object",
          "properties": {
            "Openness": {
              "type": "object",
              "properties": {
                "evidence": { "type": "string" },
                "label": { "type": "integer", "enum": [0, 1] }
              },
              "required": ["evidence", "label"]
            },
            "Conscientiousness": {
              "type": "object",
              "properties": {
                "evidence": { "type": "string" },
                "label": { "type": "integer", "enum": [0, 1] }
              },
              "required": ["evidence", "label"]
            },
            "Extraversion": {
              "type": "object",
              "properties": {
                "evidence": { "type": "string" },
                "label": { "type": "integer", "enum": [0, 1] }
              },
              "required": ["evidence", "label"]
            },
            "Agreeableness": {
              "type": "object",
              "properties": {
                "evidence": { "type": "string" },
                "label": { "type": "integer", "enum": [0, 1] }
              },
              "required": ["evidence", "label"]
            },
            "Neuroticism": {
              "type": "object",
              "properties": {
                "evidence": { "type": "string" },
                "label": { "type": "integer", "enum": [0, 1] }
              },
              "required": ["evidence", "label"]
            }
          },
          "required": ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]
        }
      },
      "required": ["dimensions"]
    }
  }
}


# For Gemini

In [ ]:
from google.ai.generativelanguage_v1beta.types import content
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_schema": content.Schema(
        type=content.Type.OBJECT,
        enum=[],
        required=["dimensions"],
        properties={
            "dimensions": content.Schema(
                type=content.Type.OBJECT,
                enum=[],
                required=["Extraversion/Introversion", "Sensing/Intuition", "Thinking/Feeling", "Judging/Perceiving"],
                properties={
                    "Extraversion/Introversion": content.Schema(
                        type=content.Type.OBJECT,
                        enum=[],
                        required=["evidence", "type"],
                        properties={
                            "evidence": content.Schema(
                                type=content.Type.STRING,
                            ),
                            "type": content.Schema(
                                type=content.Type.STRING,  # Using STRING to allow "E/I"
                                enum=["E", "I"],
                            ),
                        },
                    ),
                    "Sensing/Intuition": content.Schema(
                        type=content.Type.OBJECT,
                        enum=[],
                        required=["evidence", "type"],
                        properties={
                            "evidence": content.Schema(
                                type=content.Type.STRING,
                            ),
                            "type": content.Schema(
                                type=content.Type.STRING,  # Using STRING to allow "S/N"
                                enum=["S", "N"],
                            ),
                        },
                    ),
                    "Thinking/Feeling": content.Schema(
                        type=content.Type.OBJECT,
                        enum=[],
                        required=["evidence", "type"],
                        properties={
                            "evidence": content.Schema(
                                type=content.Type.STRING,
                            ),
                            "type": content.Schema(
                                type=content.Type.STRING,  # Using STRING to allow "T/F"
                                enum=["T", "F"],
                            ),
                        },
                    ),
                    "Judging/Perceiving": content.Schema(
                        type=content.Type.OBJECT,
                        enum=[],
                        required=["evidence", "type"],
                        properties={
                            "evidence": content.Schema(
                                type=content.Type.STRING,
                            ),
                            "type": content.Schema(
                                type=content.Type.STRING,  # Using STRING to allow "J/P"
                                enum=["J", "P"],
                            ),
                        },
                    ),
                },
            ),
        },
    ),
    "response_mime_type": "application/json",
}

## Run

In [ ]:
batch = client.batches.create(
    input_file_id=input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

In [ ]:
batch = client.batches.retrieve(batch.id)
print(batch)

In [ ]:

file_response = client.files.content(batch.output_file_id)
#print(file_response.text)

# Single

In [ ]:
    response = client.chat.completions.create(
        model="meta-llama-3.1-8b-instruct",
        messages=messages,
        response_format=response_format
    )

## Fine Tune

In [ ]:
response = client.fine_tuning.jobs.create(
  training_file=input_file.id,
  model="gpt-4o-mini-2024-07-18",
  hyperparameters={
    "n_epochs":2
  }
)

In [ ]:
fine_tune_details = client.fine_tuning.jobs.retrieve(response.id)
print(fine_tune_details)

# API Call

## OPEN_AI/ OPEN_AI like

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="_____", base_url="http://localhost:1234/v1")

# Gemini

In [ ]:
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content
genai.configure(api_key="______")

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
)

chat_session = model.start_chat(
    history= [
        
    ]
)
response = chat_session.send_message(content=input)
